1. Dataset creation and manipulation

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import warnings
warnings.filterwarnings('ignore')

## 1. Load Raw Data


In [2]:
# Load the datasets
sentiment_labeled_data = pd.read_csv('data/raw/stock_data.csv')
stock_tweets = pd.read_csv('data/raw/stock_tweets_d1.csv')
stock_prices = pd.read_csv('data/raw/stock_yfinance_data_d1.csv')

print("Dataset shapes:")
print(f"sentiment_labeled_data: {sentiment_labeled_data.shape}")
print(f"stock_tweets: {stock_tweets.shape}")
print(f"stock_prices: {stock_prices.shape}")


Dataset shapes:
sentiment_labeled_data: (5791, 2)
stock_tweets: (80793, 4)
stock_prices: (6300, 8)


## 2. Explore the Data


In [3]:
# Explore sentiment_labeled_data (labeled sentiment data)
print("\nFirst few rows:")
print(sentiment_labeled_data.head())
print("\nInfo:")
print(sentiment_labeled_data.info())
print("\nSentiment distribution:")
print(sentiment_labeled_data['Sentiment'].value_counts())
print("\nMissing values:")
print(sentiment_labeled_data.isnull().sum())



First few rows:
                                                Text  Sentiment
0  Kickers on my watchlist XIDE TIT SOQ PNK CPW B...          1
1  user: AAP MOVIE. 55% return for the FEA/GEED i...          1
2  user I'd be afraid to short AMZN - they are lo...          1
3                                  MNTA Over 12.00            1
4                                   OI  Over 21.37            1

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5791 entries, 0 to 5790
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       5791 non-null   object
 1   Sentiment  5791 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.6+ KB
None

Sentiment distribution:
Sentiment
 1    3685
-1    2106
Name: count, dtype: int64

Missing values:
Text         0
Sentiment    0
dtype: int64


In [4]:
# Explore stock_tweets (tweets with stock info)
print("\nFirst few rows:")
print(stock_tweets.head())
print("\nInfo:")
print(stock_tweets.info())
print("\nUnique stocks:")
print(stock_tweets['Stock Name'].value_counts())
print("\nMissing values:")
print(stock_tweets.isnull().sum())



First few rows:
                        Date  \
0  2022-09-29 23:41:16+00:00   
1  2022-09-29 23:24:43+00:00   
2  2022-09-29 23:18:08+00:00   
3  2022-09-29 22:40:07+00:00   
4  2022-09-29 22:27:05+00:00   

                                               Tweet Stock Name Company Name  
0  Mainstream media has done an amazing job at br...       TSLA  Tesla, Inc.  
1  Tesla delivery estimates are at around 364k fr...       TSLA  Tesla, Inc.  
2  3/ Even if I include 63.0M unvested RSUs as of...       TSLA  Tesla, Inc.  
3  @RealDanODowd @WholeMarsBlog @Tesla Hahaha why...       TSLA  Tesla, Inc.  
4  @RealDanODowd @Tesla Stop trying to kill kids,...       TSLA  Tesla, Inc.  

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80793 entries, 0 to 80792
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          80793 non-null  object
 1   Tweet         80793 non-null  object
 2   Stock Name    80793 non-null

Date            0
Tweet           0
Stock Name      0
Company Name    0
dtype: int64


In [5]:
# Explore stock_prices (yfinance data)
print("\nFirst few rows:")
print(stock_prices.head())
print("\nInfo:")
print(stock_prices.info())
print("\nDate range:")
stock_prices['Date'] = pd.to_datetime(stock_prices['Date'])
print(f"From {stock_prices['Date'].min()} to {stock_prices['Date'].max()}")
print("\nUnique stocks:")
print(stock_prices['Stock Name'].value_counts())
print("\nMissing values:")
print(stock_prices.isnull().sum())



First few rows:
         Date        Open        High         Low       Close   Adj Close  \
0  2021-09-30  260.333344  263.043335  258.333344  258.493347  258.493347   
1  2021-10-01  259.466675  260.260010  254.529999  258.406677  258.406677   
2  2021-10-04  265.500000  268.989990  258.706665  260.510010  260.510010   
3  2021-10-05  261.600006  265.769989  258.066681  260.196655  260.196655   
4  2021-10-06  258.733337  262.220001  257.739990  260.916656  260.916656   

     Volume Stock Name  
0  53868000       TSLA  
1  51094200       TSLA  
2  91449900       TSLA  
3  55297800       TSLA  
4  43898400       TSLA  

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6300 entries, 0 to 6299
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        6300 non-null   object 
 1   Open        6300 non-null   float64
 2   High        6300 non-null   float64
 3   Low         6300 non-null   float64
 4   Close 

## 3. Clean and Preprocess Data


In [6]:
# Text cleaning function with ticker preservation
def clean_text(text):
    """Clean and preprocess tweet text - preserves stock ticker symbols"""
    if pd.isna(text):
        return ""
    
    
    # Convert to lowercase (tickers now preserved as ticker_xxx)
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove user mentions and hashtags symbols (but keep the text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    
    # Keep letters, digits, spaces, underscores (for ticker_), and financial symbols (%, $, .)
    text = re.sub(r'[^a-z0-9\s%$._€]', '', text)
    
    # Remove extra whitespace
    text = ' '.join(text.split())
    
    return text

print("Text cleaning function with ticker preservation defined!")


Text cleaning function with ticker preservation defined!


In [7]:
# Process sentiment_labeled_data (labeled sentiment tweets)
sentiment_labeled_data_cleaned = sentiment_labeled_data.copy()

# Clean the text
sentiment_labeled_data_cleaned['cleaned_text'] = sentiment_labeled_data_cleaned['Text'].apply(clean_text)

# Remove rows with empty cleaned text
sentiment_labeled_data_cleaned = sentiment_labeled_data_cleaned[sentiment_labeled_data_cleaned['cleaned_text'].str.len() > 0]

print(f"Original rows: {len(sentiment_labeled_data)}")
print(f"Rows after cleaning: {len(sentiment_labeled_data_cleaned)}")
print(f"\nSample cleaned tweets:")
print(sentiment_labeled_data_cleaned[['Text', 'cleaned_text', 'Sentiment']].head())


Original rows: 5791
Rows after cleaning: 5791

Sample cleaned tweets:
                                                Text  \
0  Kickers on my watchlist XIDE TIT SOQ PNK CPW B...   
1  user: AAP MOVIE. 55% return for the FEA/GEED i...   
2  user I'd be afraid to short AMZN - they are lo...   
3                                  MNTA Over 12.00     
4                                   OI  Over 21.37     

                                        cleaned_text  Sentiment  
0  kickers on my watchlist xide tit soq pnk cpw b...          1  
1  user aap movie. 55% return for the feageed ind...          1  
2  user id be afraid to short amzn they are looki...          1  
3                                    mnta over 12.00          1  
4                                      oi over 21.37          1  


In [8]:
# Process stock_tweets
stock_tweets_cleaned = stock_tweets.copy()

# Convert date column to datetime
stock_tweets_cleaned['Date'] = pd.to_datetime(stock_tweets_cleaned['Date'])

# Clean the tweet text
stock_tweets_cleaned['cleaned_tweet'] = stock_tweets_cleaned['Tweet'].apply(clean_text)

# Remove rows with empty cleaned text
stock_tweets_cleaned = stock_tweets_cleaned[stock_tweets_cleaned['cleaned_tweet'].str.len() > 0]

# Extract date only (without time) for easier merging with stock prices
stock_tweets_cleaned['date_only'] = stock_tweets_cleaned['Date'].dt.date

print(f"Original rows: {len(stock_tweets)}")
print(f"Rows after cleaning: {len(stock_tweets_cleaned)}")
print(f"\nSample cleaned tweets:")
print(stock_tweets_cleaned[['Date', 'cleaned_tweet', 'Stock Name']].head())


Original rows: 80793
Rows after cleaning: 80792

Sample cleaned tweets:
                       Date  \
0 2022-09-29 23:41:16+00:00   
1 2022-09-29 23:24:43+00:00   
2 2022-09-29 23:18:08+00:00   
3 2022-09-29 22:40:07+00:00   
4 2022-09-29 22:27:05+00:00   

                                       cleaned_tweet Stock Name  
0  mainstream media has done an amazing job at br...       TSLA  
1  tesla delivery estimates are at around 364k fr...       TSLA  
2  3 even if i include 63.0m unvested rsus as of ...       TSLA  
3  hahaha why are you still trying to stop tesla ...       TSLA  
4  stop trying to kill kids you sad deranged old man       TSLA  


In [9]:
# Process stock_prices
stock_prices_cleaned = stock_prices.copy()

# Date is already converted to datetime in previous cell
# Calculate additional features
stock_prices_cleaned['daily_return'] = stock_prices_cleaned.groupby('Stock Name')['Close'].pct_change()
stock_prices_cleaned['price_range'] = stock_prices_cleaned['High'] - stock_prices_cleaned['Low']
stock_prices_cleaned['date_only'] = stock_prices_cleaned['Date'].dt.date

# Handle any missing values
stock_prices_cleaned = stock_prices_cleaned.dropna()

print(f"Original rows: {len(stock_prices)}")
print(f"Rows after processing: {len(stock_prices_cleaned)}")
print(f"\nSample processed data:")
print(stock_prices_cleaned.head())


Original rows: 6300
Rows after processing: 6275

Sample processed data:
        Date        Open        High         Low       Close   Adj Close  \
1 2021-10-01  259.466675  260.260010  254.529999  258.406677  258.406677   
2 2021-10-04  265.500000  268.989990  258.706665  260.510010  260.510010   
3 2021-10-05  261.600006  265.769989  258.066681  260.196655  260.196655   
4 2021-10-06  258.733337  262.220001  257.739990  260.916656  260.916656   
5 2021-10-07  261.820007  268.333344  261.126678  264.536682  264.536682   

     Volume Stock Name  daily_return  price_range   date_only  
1  51094200       TSLA     -0.000335     5.730011  2021-10-01  
2  91449900       TSLA      0.008140    10.283325  2021-10-04  
3  55297800       TSLA     -0.001203     7.703308  2021-10-05  
4  43898400       TSLA      0.002767     4.480011  2021-10-06  
5  57587400       TSLA      0.013874     7.206665  2021-10-07  


## 4. Merge Datasets (Tweets + Stock Prices)


In [10]:
# Merge stock tweets with stock prices based on date and stock name
merged_data = pd.merge(
    stock_tweets_cleaned,
    stock_prices_cleaned,
    left_on=['date_only', 'Stock Name'],
    right_on=['date_only', 'Stock Name'],
    how='inner'
)

# Select relevant columns
merged_data = merged_data[[
    'Date_x', 'cleaned_tweet', 'Stock Name', 'Company Name',
    'Open', 'High', 'Low', 'Close', 'Volume', 'daily_return', 'price_range'
]]

# Rename Date_x to Date
merged_data = merged_data.rename(columns={'Date_x': 'Date'})

print(f"Merged dataset shape: {merged_data.shape}")
print(f"\nSample merged data:")
print(merged_data.head())
print(f"\nDate range: {merged_data['Date'].min()} to {merged_data['Date'].max()}")


Merged dataset shape: (63497, 11)

Sample merged data:
                       Date  \
0 2022-09-29 23:41:16+00:00   
1 2022-09-29 23:24:43+00:00   
2 2022-09-29 23:18:08+00:00   
3 2022-09-29 22:40:07+00:00   
4 2022-09-29 22:27:05+00:00   

                                       cleaned_tweet Stock Name Company Name  \
0  mainstream media has done an amazing job at br...       TSLA  Tesla, Inc.   
1  tesla delivery estimates are at around 364k fr...       TSLA  Tesla, Inc.   
2  3 even if i include 63.0m unvested rsus as of ...       TSLA  Tesla, Inc.   
3  hahaha why are you still trying to stop tesla ...       TSLA  Tesla, Inc.   
4  stop trying to kill kids you sad deranged old man       TSLA  Tesla, Inc.   

        Open        High         Low       Close    Volume  daily_return  \
0  282.76001  283.649994  265.779999  268.209991  77620600     -0.068101   
1  282.76001  283.649994  265.779999  268.209991  77620600     -0.068101   
2  282.76001  283.649994  265.779999  268.209991 

## 5. Save Processed Data


In [15]:
# Save processed datasets to the processed folder
sentiment_labeled_data_cleaned.to_csv('data/processed/labeled_sentiment_tweets.csv', index=False)
stock_tweets_cleaned.to_csv('data/processed/cleaned_stock_tweets.csv', index=False)
stock_prices_cleaned.to_csv('data/processed/stock_prices_features.csv', index=False)
merged_data.to_csv('data/processed/tweets_with_stock_data.csv', index=False)

## 6. Data Summary & Next Steps


In [13]:
# Summary of processed data
print("\n DATASET SUMMARY:")
print(f"\n1. Labeled Sentiment Tweets: {len(sentiment_labeled_data_cleaned):,} rows")
print(f"   - Purpose: Training sentiment model")
print(f"   - Sentiment distribution: {dict(sentiment_labeled_data_cleaned['Sentiment'].value_counts())}")

print(f"\n2. Cleaned Stock Tweets: {len(stock_tweets_cleaned):,} rows")
print(f"   - Purpose: Applying sentiment model to real stock data")
print(f"   - Stocks: {', '.join(stock_tweets_cleaned['Stock Name'].unique())}")

print(f"\n3. Stock Prices with Features: {len(stock_prices_cleaned):,} rows")
print(f"   - Purpose: Financial features for prediction")
print(f"   - Features: Open, High, Low, Close, Volume, daily_return, price_range")

print(f"\n4. Merged Tweets + Stock Data: {len(merged_data):,} rows")
print(f"   - Purpose: Combined dataset for sentiment-based stock prediction")
print(f"   - Ready for model training!")




 DATASET SUMMARY:

1. Labeled Sentiment Tweets: 5,791 rows
   - Purpose: Training sentiment model
   - Sentiment distribution: {1: 3685, -1: 2106}

2. Cleaned Stock Tweets: 80,792 rows
   - Purpose: Applying sentiment model to real stock data
   - Stocks: TSLA, MSFT, PG, META, AMZN, GOOG, AMD, AAPL, NFLX, TSM, KO, F, COST, DIS, VZ, CRM, INTC, BA, BX, NOC, PYPL, ENPH, NIO, ZS, XPEV

3. Stock Prices with Features: 6,275 rows
   - Purpose: Financial features for prediction
   - Features: Open, High, Low, Close, Volume, daily_return, price_range

4. Merged Tweets + Stock Data: 63,497 rows
   - Purpose: Combined dataset for sentiment-based stock prediction
   - Ready for model training!
